In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

In [5]:
excel_file = 'MasterData.xlsx'
bitcoin_df = pd.read_excel(excel_file, sheet_name='Bitcoin Historical Data')
ethereum_df = pd.read_excel(excel_file, sheet_name='Ethereum Historical Data')
oil_df = pd.read_excel(excel_file, sheet_name='DCOILWTICO')
vix_df = pd.read_excel(excel_file, sheet_name='VIXCLS')
interest_df = pd.read_excel(excel_file, sheet_name='DFF')
nasdaq_df = pd.read_excel(excel_file, sheet_name='NASDAQCOM')
gold_df = pd.read_excel(excel_file, sheet_name='Gold Futures Historical Data')

In [6]:
def convert_volume(vol):
    if pd.isna(vol):
        return np.nan
    vol = str(vol).strip().upper()
    try:
        if 'K' in vol:
            return float(vol.replace('K', '')) * 1_000
        elif 'M' in vol:
            return float(vol.replace('M', '')) * 1_000_000
        elif 'B' in vol:
            return float(vol.replace('B', '')) * 1_000_000_000
        else:
            return float(vol)
    except:
        return np.nan

bitcoin_df['Vol.'] = bitcoin_df['Vol.'].apply(convert_volume)
ethereum_df['Vol.'] = ethereum_df['Vol.'].apply(convert_volume)
gold_df['Vol.'] = gold_df['Vol.'].apply(convert_volume)

print("Volume conversion complete")

Volume conversion complete


In [9]:
bitcoin_df = bitcoin_df.rename(columns={
    'Date': 'Date', 'Open': 'BTC_Open', 'High': 'BTC_High', 
    'Low': 'BTC_Low', 'Price': 'BTC_Close', 'Vol.': 'BTC_Volume'
})

ethereum_df = ethereum_df.rename(columns={'Date': 'Date', 'Price': 'ETH_Close'})
gold_df = gold_df.rename(columns={'Date': 'Date', 'Price': 'Gold_Close'})
oil_df = oil_df.rename(columns={'observation_date': 'Date', 'DCOILWTICO': 'Oil_WTI'})
vix_df = vix_df.rename(columns={'observation_date': 'Date', 'VIXCLS': 'VIX'})
interest_df = interest_df.rename(columns={'observation_date': 'Date', 'DFF': 'Interest_Rate'})
nasdaq_df = nasdaq_df.rename(columns={'observation_date': 'Date', 'NASDAQCOM': 'NASDAQ'})

print("Columns renamed")

Columns renamed


In [11]:
bitcoin_clean = bitcoin_df[['Date', 'BTC_Open', 'BTC_High', 'BTC_Low', 'BTC_Close', 'BTC_Volume']]
ethereum_clean = ethereum_df[['Date', 'ETH_Close']]
gold_clean = gold_df[['Date', 'Gold_Close']]
oil_clean = oil_df[['Date', 'Oil_WTI']]
vix_clean = vix_df[['Date', 'VIX']]
interest_clean = interest_df[['Date', 'Interest_Rate']]
nasdaq_clean = nasdaq_df[['Date', 'NASDAQ']]

print("Selected columns")

Selected columns


In [13]:
master_df = bitcoin_clean.copy()
master_df = pd.merge(master_df, ethereum_clean, on='Date', how='left')
master_df = pd.merge(master_df, gold_clean, on='Date', how='left')
master_df = pd.merge(master_df, oil_clean, on='Date', how='left')
master_df = pd.merge(master_df, vix_clean, on='Date', how='left')
master_df = pd.merge(master_df, interest_clean, on='Date', how='left')
master_df = pd.merge(master_df, nasdaq_clean, on='Date', how='left')
master_df = master_df.sort_values('Date').reset_index(drop=True)

print(f"Merged dataset: {master_df.shape}")
master_df.head()

Merged dataset: (2131, 12)


,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,ETH_Close,Gold_Close,Oil_WTI,VIX,Interest_Rate,NASDAQ
0,2020-01-01,7196.4,7259.4,7180.0,7199.8,420280.0,130.75,NaN,NaN,NaN,1.55,NaN
1,2020-01-02,7199.7,7209.6,6901.4,6967.0,632780.0,127.19,1528.1,61.17,12.47,1.55,9092.19
2,2020-01-03,6967.1,7402.9,6884.1,7343.1,936290.0,134.27,1552.4,63.00,14.02,1.55,9020.77
3,2020-01-04,7345.1,7433.1,7291.4,7376.8,523910.0,134.33,NaN,NaN,NaN,1.55,NaN
4,2020-01-05,7376.8,7501.0,7345.6,7372.5,628140.0,135.59,NaN,NaN,NaN,1.55,NaN


In [15]:
print("Missing values BEFORE cleaning:")
print(master_df.isnull().sum())

Missing values BEFORE cleaning:
Date               0
BTC_Open           0
BTC_High           0
BTC_Low            0
BTC_Close          0
BTC_Volume         0
ETH_Close          0
Gold_Close       626
Oil_WTI          671
VIX              638
Interest_Rate      0
NASDAQ           664
dtype: int64


In [17]:
columns_to_fill = [col for col in master_df.columns if col != 'Date']
master_df[columns_to_fill] = master_df[columns_to_fill].ffill()
master_df[columns_to_fill] = master_df[columns_to_fill].bfill()

print("Missing values AFTER cleaning:")
print(master_df.isnull().sum())

Missing values AFTER cleaning:
Date             0
BTC_Open         0
BTC_High         0
BTC_Low          0
BTC_Close        0
BTC_Volume       0
ETH_Close        0
Gold_Close       0
Oil_WTI          0
VIX              0
Interest_Rate    0
NASDAQ           0
dtype: int64


In [19]:
dates = master_df['Date'].copy()
feature_columns = [col for col in master_df.columns if col != 'Date']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(master_df[feature_columns])
scaled_df = pd.DataFrame(scaled_features, columns=feature_columns, index=master_df.index)
scaled_df.insert(0, 'Date', dates)

print("Normalization complete!")
print(f"Shape: {scaled_df.shape}")

Normalization complete!
Shape: (2131, 12)


In [21]:
print("CLEANED DATA (original scale):")
master_df.head()

CLEANED DATA (original scale):


,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,ETH_Close,Gold_Close,Oil_WTI,VIX,Interest_Rate,NASDAQ
0,2020-01-01,7196.4,7259.4,7180.0,7199.8,420280.0,130.75,1528.1,61.17,12.47,1.55,9092.19
1,2020-01-02,7199.7,7209.6,6901.4,6967.0,632780.0,127.19,1528.1,61.17,12.47,1.55,9092.19
2,2020-01-03,6967.1,7402.9,6884.1,7343.1,936290.0,134.27,1552.4,63.00,14.02,1.55,9020.77
3,2020-01-04,7345.1,7433.1,7291.4,7376.8,523910.0,134.33,1552.4,63.00,14.02,1.55,9020.77
4,2020-01-05,7376.8,7501.0,7345.6,7372.5,628140.0,135.59,1552.4,63.00,14.02,1.55,9020.77


In [23]:
print("SCALED DATA (normalized):")
scaled_df.head()

SCALED DATA (normalized):


,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,ETH_Close,Gold_Close,Oil_WTI,VIX,Interest_Rate,NASDAQ
0,2020-01-01,-1.252510,-1.261630,-1.241000,-1.253150,-0.119386,-1.696053,-1.095486,-0.498412,-1.081729,-0.523329,-1.525582
1,2020-01-02,-1.252403,-1.263220,-1.250173,-1.260690,-0.118486,-1.699065,-1.095486,-0.498412,-1.081729,-0.523329,-1.525582
2,2020-01-03,-1.259942,-1.257048,-1.250743,-1.248508,-0.117201,-1.693074,-1.051908,-0.402306,-0.884577,-0.523329,-1.546371
3,2020-01-04,-1.247690,-1.256083,-1.237332,-1.247416,-0.118947,-1.693024,-1.051908,-0.402306,-0.884577,-0.523329,-1.546371
4,2020-01-05,-1.246663,-1.253915,-1.235547,-1.247556,-0.118506,-1.691958,-1.051908,-0.402306,-0.884577,-0.523329,-1.546371


In [25]:
master_df.describe()

,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,ETH_Close,Gold_Close,Oil_WTI,VIX,Interest_Rate,NASDAQ
count,2131,2131.000000,2131.000000,2131.000000,2131.000000,2.131000e+03,2131.000000,2131.000000,2131.000000,2131.000000,2131.000000,2131.000000
mean,2022-12-01 00:00:00,45839.056546,46768.984467,44869.531675,45887.046363,2.862292e+07,2135.322088,2138.956842,70.660479,20.974524,2.723951,14333.156373
min,2020-01-01 00:00:00,4815.200000,5369.300000,3869.500000,4826.000000,2.600000e+02,107.900000,1477.900000,-36.980000,11.860000,0.040000,6860.670000
25%,2021-06-16 12:00:00,21453.200000,21834.350000,20914.800000,21503.550000,5.577000e+04,1384.570000,1793.500000,62.910000,15.855000,0.090000,11716.680000
50%,2022-12-01 00:00:00,38688.200000,39838.500000,37556.000000,38709.700000,9.049000e+04,2004.770000,1916.500000,72.210000,19.080000,3.830000,13814.490000
75%,2024-05-16 12:00:00,62926.100000,63916.000000,61352.350000,62937.900000,2.169450e+05,3056.215000,2336.600000,80.755000,24.200000,4.830000,16379.460000
max,2025-10-31 00:00:00,124687.500000,126186.000000,123144.600000,124725.100000,4.470000e+09,4831.210000,4359.400000,123.640000,82.690000,5.330000,23958.470000
std,NaN,30859.418896,31323.652893,30377.427468,30879.255225,2.362871e+08,1182.181817,557.743346,19.045916,7.863820,2.243765,3436.194843


In [29]:
print("master_df shape:", master_df.shape)
print("scaled_df shape:", scaled_df.shape)
print("feature_columns:", feature_columns)

master_df shape: (2131, 12)
scaled_df shape: (2131, 12)
feature_columns: ['BTC_Open', 'BTC_High', 'BTC_Low', 'BTC_Close', 'BTC_Volume', 'ETH_Close', 'Gold_Close', 'Oil_WTI', 'VIX', 'Interest_Rate', 'NASDAQ']
